# MindBridge: Connecting the Brain to ChatGPT
### Unified Cognitive–Emotional–BCI AI System

This notebook integrates three research streams:
1. **Cognitive-to-Linguistic Interface**  
2. **Emotionally Adaptive ChatGPT**  
3. **Brain-Controlled ChatGPT Interface**

We use EEG data from two datasets:
- PhysioNet EEG Motor/Imagery  [(Resource)](https://physionet.org/content/eegmmidb/1.0.0/?utm_source=chatgpt.com)
- EEG Emotion Classification Dataset  [(Resource)](https://zenodo.org/records/14787743?utm_source=chatgpt.com)

The system extracts EEG features, maps them to emotional/cognitive states, and dynamically adjusts ChatGPT prompts for context-aware, adaptive responses.

## Setup & Installation
In this cell, we install all dependencies required for EEG preprocessing, model building, and API integration.

In [1]:
!pip install mne numpy pandas matplotlib scikit-learn tensorflow keras openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.3 MB/s eta 0:00:00


## Import All necessary Libraries
Import all key libraries for EEG data manipulation, visualization, and model building

In [2]:
import os
import numpy as np
import pandas as pd
import mne
import shutil
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import openai


## Upload DataSets and UnZip

In [ ]:
# Define your zip file paths
physionet_zip = "/content/drive/MyDrive/data/PhysioNet_EEG.zip"
emotion_zip = "/content/drive/MyDrive/data/EEG_Emotion_Dataset.zip"

# Define your target extraction directories
physionet_folder = "/content/data/physionet"
emotion_folder = "/content/data/emotion_eeg"

# Create directories if they don't exist
os.makedirs(physionet_folder, exist_ok=True)
os.makedirs(emotion_folder, exist_ok=True)

# --- Unzip PhysioNet dataset ---
shutil.unpack_archive(physionet_zip, physionet_folder)
print("✅ PhysioNet dataset unzipped successfully at:", physionet_folder)

# --- Unzip Emotion EEG dataset ---
shutil.unpack_archive(emotion_zip, emotion_folder)
print("✅ Emotion EEG dataset unzipped successfully at:", emotion_folder)


## Load PhysioNet EEG Data
Load and visualize an EEG sample from PhysioNet.
This dataset will be used for cognitive and motor-intent decoding.

In [ ]:
edf_file = os.path.join(PHYSIONET_PATH, "S001R01.edf")  # example file
raw = mne.io.read_raw_edf(edf_file, preload=True)
raw.filter(0.5, 45)

print(raw.info)
raw.plot(duration=5, n_channels=10)


## Extract Cognitive Features
Convert EEG signals to frequency-domain features such as Power Spectral Density (PSD) and Differential Entropy (DE), which represent mental activity patterns.

In [ ]:
def extract_features(eeg_data):
    psd, freqs = mne.time_frequency.psd_welch(eeg_data, fmin=0.5, fmax=45, n_fft=256)
    return np.mean(psd, axis=1)

cognitive_features = extract_features(raw)
print("Feature shape:", cognitive_features.shape)


## Load Emotion EEG Dataset
Now, we load the second dataset — EEG Emotion Classification (Biosemi or Muse2).
This dataset helps train the Emotionally Adaptive component.

In [ ]:
emotion_file = os.path.join(EMOTION_PATH, "participant_01.csv")
emotion_data = pd.read_csv(emotion_file)

print("Columns:", emotion_data.columns)
emotion_data.head()


## Emotion Feature Preprocessing
Normalize EEG features and encode emotion labels (happy, sad, calm, angry).

In [ ]:
X = emotion_data.drop(columns=['label'])
y = emotion_data['label']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Preprocessed Emotion Data Shape:", X_scaled.shape)


## Build Emotion Classification Model
Train a simple neural network for emotion recognition.
Later, we’ll use its output to condition ChatGPT’s tone and response style.

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_scaled, y_encoded, epochs=15, batch_size=32, validation_split=0.2)


## Multimodal Feature Fusion (M3DM Layer)
Combine the cognitive features and emotion features into a unified embedding vector for adaptive prompt engineering.

In [ ]:
def feature_fusion(cognitive_vector, emotion_vector):
    combined = np.concatenate((cognitive_vector, np.mean(emotion_vector, axis=0)))
    return combined / np.linalg.norm(combined)

fused_features = feature_fusion(cognitive_features, X_scaled)
print("Fused feature vector shape:", fused_features.shape)


## Adaptive Prompt Engineering
Map the fused embeddings to adaptive ChatGPT prompt modifications.
The emotional tone and cognitive signals determine response structure and complexity.

In [ ]:
def generate_adaptive_prompt(emotion_label, cognitive_state):
    if emotion_label == "happy":
        style = "enthusiastic and supportive"
    elif emotion_label == "sad":
        style = "calm and encouraging"
    else:
        style = "neutral and informative"

    if cognitive_state == "focused":
        complexity = "technical and detail-oriented"
    else:
        complexity = "simplified and conversational"

    return f"Respond in a {style} tone with {complexity} explanations."

example_prompt = generate_adaptive_prompt("happy", "focused")
print(example_prompt)
